In [ ]:
# Utilizzando il modulo random, realizza il gioco del sasso, carta, forbici:
# - sasso vince su forbici
# - carta vince su sasso
# - forbici vince su carta
# - ogni simbolo pareggia contro ogni simbolo uguale

# Implementate l'algoritmo:
# - definendo una funzione per giocare
# - permettendo all'utente di scegliere tra sasso, carta e forbici
# - registrando il numero di vittorie, pareggi e sconfitte
# - continuando a far giocare l'utente fino a quando non decide di smettere

# BONUS: registra i dati della partita su un file di testo, decidendo quale sia il formato migliore

# BONUS 2: prevedi e testa una funzione per poter leggere da un file di testo i dati di una partita e visualizzarli

# BONUS 3: i dati di ciascuna partita devono essere salvati in un file txt che riporta la data 
# e l'ora della fine della partita (ad esempio se la partita finisce il 14/06/2024 alle 20:02, dovrà chiamarsi
#                                   "2024_06_14_20_02.txt")

# BONUS 4: prevedi una funzione per poter accedere a tutti i dati delle partite salvate e fare una statistica dei giocatori.
#          Prevedi quindi di chiedere il nickname del giocatore prima di avviare la partita. Il nickname dovrà comparire 
#          tra i dati del .txt 

In [23]:
import datetime
import random
import pandas as pd
import os 

lista_figure = ["sasso", "carta", "forbici"]
dataCorrente = datetime.datetime.now()
data_str = dataCorrente.strftime('%m_%d_%y    %H_%M') ## perchè se utilizzo gli underscore per la data non posso utillizare i due punti per l'ora????
nome_file1=data_str+ ".csv"
nick=input("Inserisci il nickname") 


def gioco():
    vittorie = 0
    pareggi = 0
    sconfitte = 0
    continua = True  
    
    while continua:
        while True:
            scegli_simbolo = input("Scegli una figura tra carta, sasso e forbici: ")
            if scegli_simbolo in lista_figure:
                break
            else:
                print("Inserisci la figura corretta!")
        
        figura_computer = random.choice(lista_figure)
        print(f"Tu hai scelto: {scegli_simbolo}")
        print(f"Il computer ha scelto: {figura_computer}")
        
        if scegli_simbolo == figura_computer:
            print("Pareggio!")
            pareggi += 1
        elif (scegli_simbolo == "sasso" and figura_computer == "forbici") or \
             (scegli_simbolo == "carta" and figura_computer == "sasso") or \
             (scegli_simbolo == "forbici" and figura_computer == "carta"):
            print("Hai vinto!")
            vittorie += 1
        else:
            print("Hai perso!")
            sconfitte += 1
        
        while True:
            risposta = input("Vuoi continuare? (si/no): ").lower()
            if risposta == "si":
                continua = True
                break
            elif risposta == "no":
                continua = False
                break
            else:
                print("Devi scegliere se continuare digitando 'si' o 'no'.")

    salva(nick + ".csv",data_str,vittorie, pareggi, sconfitte)
    df=leggi_csv(nick + ".csv")
    #print(df["vittorie"].sum())
    vittorie_totali=df["vittorie"].sum()
    sconfitte_totali=df["sconfitte"].sum()
    pareggi_totali=df["pareggi"].sum()
    
    #DataFrame con colonne nickname,vittorie totali, sconfitte totali e pareggi totali e come righe avra una riga per ogni utente
    #nickname, vittorie totali,sconfitte totale,pareggi totali
    #pippo, 10,2,3
    #pluto, 5,3,4
    aggiorna_classifica(path="classifica.csv",nick=nick,
                        vittorie_totali=vittorie_totali,
                        pareggi_totali=pareggi_totali,
                        sconfitte_totali=sconfitte_totali,)
    
    
    classifica=pd.read_csv("classifica.csv",index_col=0)
    
   
    
    
    

def salva(path,data_str,vittorie, pareggi, sconfitte):
    data={
        "data":[data_str],
        "vittorie":[vittorie],
        "sconfitte":[sconfitte],
        "pareggi":[pareggi]
    }
    df=pd.DataFrame(data=data)
    df.set_index("data",inplace=True)
    if os.path.isfile(path=path):
        df.to_csv(path, mode="a",header=False)
    else:
        df.to_csv(path)        

def leggi_csv(path):
    df=pd.read_csv(filepath_or_buffer=path,sep=",")
    return df

def aggiorna_classifica(path,nick,vittorie_totali,pareggi_totali,sconfitte_totali):
    data={
        "nick":[],
        "Vittorie Totali":[],
        "Sconfitte Totali":[],
        "Pareggi Totali":[]
    }
    df=pd.DataFrame(data=data)
    df.set_index("nick",inplace=True)
    df.loc[nick]=[vittorie_totali,sconfitte_totali,pareggi_totali]
    if os.path.isfile(path=path):
        df.to_csv(path, mode="a",header=False)
    else:
        df.to_csv(path,mode="a",header=True)
        

gioco()




    



Tu hai scelto: carta
Il computer ha scelto: sasso
Hai vinto!
